# Jupyter Notebook if parts of the App for testing

In [1]:
# LOADING
import os
import json
import openai
from llama_index import SimpleDirectoryReader


with open("../CONFIG_LIST.json", "r") as file:
    config = json.load(file)

os.environ['OPENAI_API_KEY'] = config["openai_api_key"]
model = config["model"]
### the bellow is need if using openAI API through Azure
# os.environ['OPENAI_API_BASE'] = config["openai_api_base"]
# os.environ['OPENAI_API_TYPE']  = config["openai_api_type"]
# os.environ['OPENAI_API_VERSION'] = config["openai_api_version"]

# Configure OpenAI API
# openai.api_type =  config["openai_api_type"]
# openai.api_version =  config["openai_api_version"]
# openai.api_base = os.environ['OPENAI_API_BASE']
# openai.api_key = os.environ['OPENAI_API_KEY']

openai.organization = config["openai_organization"]
openai.api_key = os.getenv("OPENAI_API_KEY")


documents = SimpleDirectoryReader('../documentS').load_data() # reads whole drectory.

documents

[Document(id_='e1b6d341-12f8-41e3-9ba4-f403a2329b7e', embedding=None, metadata={'page_label': '1', 'file_name': 'Exploratory_Analysis_to_Address_Deep_Uncertainty__Using_Calibratable_System_Models_for_Exploratory_Simulation_of_Complex_Missions[.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7ebac01e725bc1337732901bcb804f6f86575d38e9f79bdd43fe2b56388b0ef6', text=' SpringSim’20, May 19-May 21, 2020,  Fairfax, VA, USA; ©2020 Society for Modeling & Simulation International (SCS)   EXPLORATORY ANALYSIS TO ADDRESS DEEP UNCERTAINTY – USING CALIBRATABLE SYSTEM MODELS FOR EXPLORATORY SIMULATION OF COMPLEX MISSIONS   Andreas Tolk Khuong Dinh  Kevin Comer Steve Scott  The MITRE Corporation The MITRE Corporation 903 Enterprise Parkway #200 Hampton, VA 23666, United States 7515 Colshire Drive McLean, VA 22102, United States {atolk, kdinh}@mitre.org {ktcomer, slscott}@mitre.org  ABSTRACT Traditional optimization methods and sometimes even the underlayin

# Using OpenAI API directly

In [3]:
from llama_index import LLMPredictor, PromptHelper, ServiceContext
from llama_index import GPTVectorStoreIndex, StorageContext, load_index_from_storage
from langchain.chat_models import ChatOpenAI


from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


def indexer(documents, model ):
    
    if os.path.exists('../storage'):
        print('index exist')
        
        # rebuild storage context
        storage_context = StorageContext.from_defaults(persist_dir="../storage")
        # load index
        index = load_index_from_storage(storage_context)

    
    else:
        print('not exist, creating index ... ')

        documents = documents
        
        llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.7, model_name=model))

        max_input_size = 4096
        num_output = 256
        max_chunk_overlap = 0.1
        chunk_size_limit = 600
        prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap,chunk_size_limit=chunk_size_limit)
        service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

        # index content in the folder documents
        index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context) 
        # Save your index to a directory called storage
        index.storage_context.persist(persist_dir="../storage")
    
    return(index)

index = indexer(documents=documents, model=model)


index exist


In [7]:

prompt = "Can you write a summary of the book 'Exploring the Depths of Human Existence' "

query_engine = index.as_query_engine()
response = query_engine.query(prompt)

print(response)

The book 'Exploring the Depths of Human Existence' is a labor of love for the author, Dr. Carlos Kuhn, and has taken over 12 years to complete. It aims to captivate readers' interest and spark excitement by delving into conversations on identity, society, and progress. The author hopes that the book will inspire thoughtful contemplation and serve as a catalyst for readers' intellectual exploration.


In [10]:

# Define tools
tools = [
    Tool(
       name = "LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True
    ),
]
#Initialize conversational memory
conversational_memory = ConversationBufferWindowMemory( memory_key='chat_history', k=5, return_messages=True )
# Initialize agent with conversational memory
agent_executor = initialize_agent(tools, llm=ChatOpenAI(temperature=0.7, model_name=model), agent="conversational-react-description", memory=conversational_memory)

# if I use my index
while True:
    prompt = input("type prompt")
    if prompt == 'thanks':
        print(f' ---------- Agent: chat is closed -------')
        break
    else:
        response = agent_executor.run(input=prompt)
        print(f'me : {prompt}')
        print(f'Agent : {response}')


me : write a summary of the book Exploring the depth of human Existence with 100 words
Agent : "Exploring the Depth of Human Existence" delves into the profound complexities of human life. It challenges readers to question their perception of reality, encouraging introspection on the nature of consciousness, human purpose, and the interplay between the self and the universe. The book presents philosophical theories, scientific research, and spiritual insights, offering a holistic view of existence. It inspires a deep understanding of our place in the cosmos, fostering a greater appreciation of the extraordinary phenomenon of life.
me : what is the name of the book?
Agent : The name of the book is "Exploring the Depth of Human Existence".
me : what was the first question I asked?
Agent : The first question you asked was, "write a summary of the book Exploring the depth of human Existence with 100 words".
 ---------- Agent: chat is closed -------


## To Do to improve the code

In [49]:
# Todo: Need to modify, to generate new index if document is added to the folder documents

import os
import hashlib

def get_directory_hash(directory):
    file_hash_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'rb') as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
                file_hash_list.append((file_path, file_hash))
    return file_hash_list

def has_directory_changed(previous_state, current_state):
    return previous_state != current_state

if __name__ == "__main__":
    directory_path = "documents"

    # Record the current state
    current_state = get_directory_hash(directory_path)

    # Compare with the previous state (you might want to load the previous state from a file)
    if has_directory_changed(previous_state, current_state):
        print("Directory has changed!")
        # Update the previous state with the current state for the next check
        previous_state = current_state
    else:
        print("Directory has not changed.")


Directory has not changed.


In [46]:
previous_state = get_directory_hash("documents")